<a href="https://colab.research.google.com/github/thibaud-perrin/hibo-mistral-7b-fc-v1/blob/main/test_hibo_mistral_7b_fc_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📖 hibo-mistral-7b-fc-v1 Testing

This notebook focuses on testing the newly saved model `thibaud-perrin/hibo-mistral-7b-fc-v1` from the Hugging Face Hub.

## 📦 Installation of Required Packages

Similar to the training notebook, we start by installing necessary packages that enable us to work with the model and perform evaluations. The `!pip install` command is used to ensure all dependencies are met for running the tests successfully.

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers sentencepiece

## 📚 Import of All Required Packages

In this step, we import all the libraries and modules required for testing the model. This includes only `transformers`.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

## 🤖 Loading the New Model and Its Tokenizer

We load the `thibaud-perrin/hibo-mistral-7b-fc-v1` model along with its tokenizer from the Hugging Face Hub. This step is crucial for preparing the model for evaluation and ensuring it can process inputs correctly.

In [ ]:
# Replace "AutoModelForCausalLM" and "AutoTokenizer" with specific model and tokenizer classes if necessary
model_identifier = "thibaud-perrin/hibo-mistral-7b-fc-v1"

model = AutoModelForCausalLM.from_pretrained(model_identifier)
tokenizer = AutoTokenizer.from_pretrained(model_identifier)

In [ ]:
tokenizer.chat_template

## 🔄 Put Model in Eval Mode

Before testing, we switch the model to evaluation mode using the `.eval()` method. This disables training-specific behaviors like dropout, ensuring the model's outputs are consistent and reflective of its true performance.

In [ ]:
model.config.use_cache = True
model.eval()

## 🧪 Test the Model

Finally, we test the model by running it on specific examples or a test dataset. This allows us to assess how well the model performs on the task it was fine-tuned for, such as instruction following or function calling. The outcomes of these tests can provide insights into any adjustments needed or confirm the model's readiness for deployment.

In [6]:
def stream(user_prompt):
    # runtimeFlag = "cuda:0"
    runtimeFlag = "cpu"
    system_prompt = """You are a helpful assistant with access to the following functions. Use them if required -
    {
        "name": "get_stock_price",
        "description": "Get the current stock price of a company",
        "parameters": {
            "type": "object",
            "properties": {
                "company_name": {
                    "type": "string",
                    "description": "The name of the company"
                },
                "exchange": {
                    "type": "string",
                    "description": "The stock exchange where the company is listed"
                }
            },
            "required": [
                "company_name",
                "exchange"
            ]
        }
    }"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt.strip()}
    ]

    transformed_data = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    print(transformed_data)

    eos_token_id = tokenizer.eos_token_id
    print(eos_token_id)
    inputs = tokenizer([transformed_data], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    _  = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=128,
        eos_token_id=eos_token_id,
        early_stopping=True,
      )

In [ ]:
stream("Hi, can you tell me the current stock price of Apple on NASDAQ?")